In [ ]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

import ribasim

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)



In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Define base_dir, results_dir and network name
base_dir = Path("..\\..\\ribasim_lumping_data\\")
results_dir = Path(base_dir, "results")
network_name = "zutphen_tki_netwerk"

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden\\wrij_afwateringseenheden_selectie_Zutphen.shp")
areas_gdf = gpd.read_file(areas_file_path)
areas_gdf = areas_gdf[['GFEIDENT', 'geometry']]

In [ ]:
# Create networkanalysis
network = RibasimLumpingNetwork(
    name=network_name, 
    results_dir=results_dir,
    areas_gdf=areas_gdf,
)

In [ ]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
);

In [ ]:
simulation_name = 'tki_zuthpen_berkel_basis.dsproj_data'
file_bc = f'{base_dir}\\d-hydro\\{simulation_name}\\FlowFM\\input\\FlowFM_boundaryconditions1d.bc'

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.get_network_data(file_bc)
# Export to geopackage
# network.export_to_geopackage()

In [ ]:
# Define locations where the network should be split into Ribasim basins:

network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kst_ST80830001', 'kst_ST80810015', # onderdoorlaten bij verdeelpunt De Berkel (Zutphen)
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151' # duikers voor wijk Zuidwijken
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
    ], 
    node_ids_to_include=[
        # 419, # voorbeeld splitsing
        # 1455, # extra punt rondom verdeelpunt De Berkel
    ],
    node_ids_to_exclude=[],
);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

Export to ribasim

In [ ]:
network.basins_gdf.head(3)

In [ ]:
network.boundaries_gdf.head(3)

In [ ]:
# network.splitnodes_moved_gdf.head(3)

In [ ]:
network.split_nodes

In [ ]:
# network.basin_connections_gdf.head(3)

nodes

In [ ]:
# set id's to node. Start with basins, then boundaries and then moved splitnodes. start with id 1
basins_gdf =network.basins_gdf.copy()
basins_gdf['node_id'] = basins_gdf['basin'] + 1

boundaries_gdf = network.boundaries_gdf.copy()
boundaries_gdf['node_id'] = boundaries_gdf['boundary_id'] + len(network.basins_gdf) +1

splitnodes_gdf = network.split_nodes.copy()
splitnodes_gdf.insert(0, 'splitnode_id', range(len(splitnodes_gdf)))
splitnodes_gdf['node_id'] = splitnodes_gdf['splitnode_id'] + len(network.basins_gdf) + len(network.boundaries_gdf) +1

In [ ]:
# concat basins, boundaries and splitnodes moved 

# oude versie ribasim
# ribasim_node_gdf = pd.concat([basins_gdf.assign(type="Basin"), boundaries_gdf.assign(type="LevelControl"),splitnodes_moved_gdf.assign(type="TabulatedRatingCurve")]) 
ribasim_node_gdf = pd.concat([basins_gdf.assign(type="Basin"), boundaries_gdf.assign(type="LevelBoundary"),splitnodes_gdf.assign(type="TabulatedRatingCurve")]) 


# set node_id as index
ribasim_node_gdf = ribasim_node_gdf.set_index('node_id')

# keep columns geometry and type
ribasim_node_gdf = ribasim_node_gdf[['geometry', 'type']]
ribasim_node_gdf

In [ ]:
# Set up the nodes:

# Make sure the feature id starts at 1: explicitly give an index.
node = ribasim.Node(
    static=ribasim_node_gdf
)
node

edges

In [ ]:
network.basin_connections_gdf.head()

In [ ]:
splitnodes_gdf

In [ ]:
basin_connections_gdf = network.basin_connections_gdf[['mesh1d_node_id', 'basin_in','basin_out','geometry']]
basin_connections_gdf = basin_connections_gdf.merge(splitnodes_gdf[['splitnode_id','mesh1d_node_id', 'node_id']], left_on='mesh1d_node_id', right_on='mesh1d_node_id')

basin_connections_gdf

In [ ]:
# basin_connections_gdf_us = network.basin_connections_gdf[['mesh1d_node_id', 'basin_out','geometry']]
basin_connections_gdf_us = basin_connections_gdf.copy()
basin_connections_gdf_us['geometry'] = basin_connections_gdf_us.geometry.apply(lambda x: LineString([x.coords[0], x.coords[1]]))
basin_connections_gdf_us['from_node_id'] = basin_connections_gdf_us['basin_out'] +1
basin_connections_gdf_us['to_node_id'] = basin_connections_gdf_us['node_id']
# basin_connections_gdf_us['to_node_id'] = basin_connections_gdf_us['splitnode_id'] + len(network.basins_gdf) + len(network.boundaries_gdf) +1
basin_connections_gdf_us.head(3)

In [ ]:
# basin_connections_gdf_ds = network.basin_connections_gdf[['mesh1d_node_id', 'basin_in','geometry']]
basin_connections_gdf_ds = basin_connections_gdf.copy()
basin_connections_gdf_ds['geometry'] = basin_connections_gdf.geometry.apply(lambda x: LineString([x.coords[1], x.coords[2]]))
# basin_connections_gdf_ds['from_node_id'] = basin_connections_gdf_ds['splitnode_moved_id'] + len(network.basins_gdf) + len(network.boundaries_gdf) +1
basin_connections_gdf_ds['from_node_id'] = basin_connections_gdf_ds['node_id']
basin_connections_gdf_ds['to_node_id'] = basin_connections_gdf_ds['basin_in'] +1
basin_connections_gdf_ds.head(3)




In [ ]:
boundary_basin_connections = network.boundary_basin_connections_gdf[['boundary_id', 'basin','geometry','boundary_location']]


# basin_connections_gdf_ds['geometry'] = basin_connections_gdf_ds.geometry.apply(lambda x: LineString([x.coords[1], x.coords[2]]))
boundary_basin_connections_us = boundary_basin_connections.loc[boundary_basin_connections['boundary_location'] == 'upstream']
boundary_basin_connections_us['from_node_id'] = boundary_basin_connections_us['boundary_id']  + len(network.basins_gdf) +1
boundary_basin_connections_us['to_node_id'] = boundary_basin_connections_us['basin'] +1

boundary_basin_connections_ds = boundary_basin_connections.loc[boundary_basin_connections['boundary_location'] == 'downstream']
boundary_basin_connections_ds['from_node_id'] = boundary_basin_connections_ds['basin'] +1
boundary_basin_connections_ds['to_node_id'] = boundary_basin_connections_ds['boundary_id'] + len(network.basins_gdf) +1

boundary_basin_connections_us.head(3)

In [ ]:
# edges = gpd.GeoDataFrame(
#     index=np.arange(0,100),
#     columns=['basin_in', 'basin_out', 'split_node', 'boundary', 'geometry']
# )
# edges['boundary'] = edges['boundary'] + 47
# edges['split_node'] = edges['split_node'] + 52
# edges#.reset_index()

In [ ]:
# network.basin_connections_gdf.geometry.apply(lambda x: LineString([x.coords[0], x.coords[1]]))

In [ ]:
# Setup the edges:
ribasim_edges = pd.concat([basin_connections_gdf_ds, basin_connections_gdf_us,boundary_basin_connections_us, boundary_basin_connections_ds]) 
ribasim_edges = ribasim_edges[['from_node_id','to_node_id','geometry']].reset_index()
ribasim_edges['from_node_id'].astype(int)

edge = ribasim.Edge(
    static=ribasim_edges
)

ribasim_edges.head(3)

basin

In [ ]:
# # Setup the basins:

# profile = pd.DataFrame(
#     data={
#         "node_id": [0, 0],
#         "storage": [0.0, 1000.0],
#         "area": [0.0, 1000.0],
#         "level": [0.0, 1.0],
#     }
# )
# repeat = np.tile([0, 1], 4)
# profile = profile.iloc[repeat]
# profile["node_id"] = [1, 1, 3, 3, 6, 6, 9, 9]

# # Convert steady forcing to m/s
# # 2 mm/d precipitation, 1 mm/d evaporation
# seconds_in_day = 24 * 3600
# precipitation = 0.002 / seconds_in_day
# evaporation = 0.001 / seconds_in_day


# static = pd.DataFrame(
#     data={
#         "node_id": [0],
#         "drainage": [0.0],
#         "potential_evaporation": [evaporation],
#         "infiltration": [0.0],
#         "precipitation": [precipitation],
#         "urban_runoff": [0.0],
#     }
# )
# static = static.iloc[[0, 0, 0, 0]]
# static["node_id"] = [1, 3, 6, 9]

# basin = ribasim.Basin(profile=profile, static=static)

In [ ]:
profile_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Basin'].index.values.tolist()
    }
)

profile_data['storage'] = 3.5
profile_data['area'] = 4.5
profile_data['level'] = 5.5

profile_data.head()


In [ ]:
static_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Basin'].index.values.tolist()
    }
)

static_data['drainage'] = 6.5
static_data['potential_evaporation'] = 6.5
static_data['infiltration'] = 6.5
static_data['precipitation'] = 6.5
static_data['urban_runoff'] = 6.5


static_data.head()

In [ ]:
basin = ribasim.Basin(profile=profile_data, static=static_data)

rating curve

In [ ]:
# Discharge: lose 1% of storage volume per day at storage = 1000.0.
static_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='TabulatedRatingCurve'].index.values.tolist()
    }
)

static_data['level'] = 6.5
static_data['discharge'] = 6.5




rating_curve = ribasim.TabulatedRatingCurve(
    static= static_data,)


static_data.head()

boundary

In [ ]:
static_boundary = boundaries_gdf[['node_id']].copy()
# static_boundary = static_boundary.rename(columns={"boundary_id": "node_id"})
static_boundary
static_boundary['level'] = 6.5

static_boundary

In [ ]:
# boundaries_gdf

In [ ]:
level_boundary = ribasim.LevelBoundary(
    static=static_boundary
)

Export everything to geopackage

In [ ]:
network.export_to_geopackage()

In [ ]:
# Setup a model:

model = ribasim.Model(
    modelname="ribasim_model",
    node=node,
    edge=edge,
    basin=basin,
    level_boundary=level_boundary,
    # level_control=level_control,
    # linear_level_connection=linear_connection,
    tabulated_rating_curve=rating_curve,
    # fractional_flow=fractional_flow,
    starttime="2020-01-01 00:00:00",
    endtime="2021-01-01 00:00:00",
)

# %%
# Write the model to a TOML and GeoPackage:

model.write(f"{results_dir}/{network.name}")

In [ ]:
# ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Basin']

In [ ]:
ribasim_node_gdf

node

In [ ]:
ribasim_node_gdf

In [ ]:
basin